In [1]:
import numpy as np
import pandas as pd
import geopandas

This notebook computes the NSE and cyclostationary NSE for each basin and each dataset combination. 

Basin shapes have been downloaded from the [GRDC website](https://www.bafg.de/GRDC/EN/02_srvcs/22_gslrs/221_MRB/riverbasins_2020.html?nn=201570#doc2731742bodyText2). The following table contains all basins with an area larger than $65,000 \ km^2$. It additionally indicates the continent where the basin is located, the ocean where water is discharged (if applicable), the basin discharge area, the main climate zone (with full names and abbreviations following the [Koppen-Geiger classification](http://worldmap.harvard.edu/data/geonode:koppen_geiger_climate_classification_rev_g21)), the area covered by the main climate zone, and the cluster determined by the hierarchical algorithm (see the dendrogram in the article). 

In [2]:
basins=geopandas.read_file('../data/basins/basins.shp').set_index('name')
basins.head()

,continent,ocean,area,climate,clim area,cluster,clim name,geometry
name,,,,,,,,
WINISK,"North America, Central America and the Caribbean",Arctic Ocean,65635.3,Dfc,99.743367,other,Snow fully humid cool summer,"POLYGON ((-87.24167 52.32500, -87.24223 52.314..."
COPPER,"North America, Central America and the Caribbean",Pacific Ocean,65704.3,Dfc,56.077317,other,Snow fully humid cool summer,"POLYGON ((-140.15833 61.00000, -140.15890 60.9..."
NOTTAWAY,"North America, Central America and the Caribbean",Arctic Ocean,66053.1,Dfc,96.210946,other,Snow fully humid cool summer,"MULTIPOLYGON (((-73.85417 50.00833, -73.85089 ..."
TALAS,Asia,---,66179.8,BSk,65.765573,other,Arid Steppe cold,"POLYGON ((73.61250 42.33333, 73.60612 42.33277..."
MARONI,South America,Atlantic Ocean,66408.4,Afm,92.235931,other,Equatorial rainforest/monsoon,"POLYGON ((-53.08750 3.20833, -53.08508 3.20798..."


All datasets available

In [3]:
list_data_P=['CPC','CRU','ERA5_Land','GLDAS20','GPCC','GPCP','GPM','JRA55','MERRA2','MSWEP','TRMM']
list_data_ET=['ERA5_Land','FLUXCOM','GLEAM','GLDAS20_NOAH36','GLDAS20_CLSM25','GLDAS20_VIC412',
              'GLDAS21_NOAH36','GLDAS21_CLSM25','GLDAS21_VIC412','GLDAS22_CLSM25','JRA55','MERRA2','MOD16','SSEBop']
list_data_R=['ERA5_Land','JRA55','GLDAS20_NOAH36','GLDAS20_CLSM25','GLDAS20_VIC412',
             'GLDAS21_NOAH36','GLDAS21_CLSM25','GLDAS21_VIC412','GLDAS22_CLSM25','GRUN','MERRA2']
data_TWS='GRACE_CSR_mascons' # 'GRACE_CSR_mascons' or 'GRACE_JPL_mascons'

The common period where all datasets are available is February 2003 - December 2014. We remove the first and last month due to the time filtering performed.

In [4]:
time_idx=pd.date_range('2003-03-01','2014-11-01',freq='MS')

Combinations of all datasets

In [5]:
list_combinations=[]
for d_P in list_data_P:
    for d_ET in list_data_ET:
        for d_R in list_data_R:
            list_combinations.append('P_{}_ET_{}_R_{}_TWS_{}'.format(d_P,d_ET,d_R,data_TWS))
print('There are {} combinations under study'.format(len(list_combinations)))

There are 1694 combinations under study


The results table contains a NSE per basin (row) and dataset combination (column).

In [6]:
df=pd.DataFrame(np.nan*np.ones((basins.shape[0],len(list_combinations))),
                    index=basins.index,columns=list_combinations)
NSE_tab=df.copy()
NSE_cyclic_tab=df.copy()

For each combination of P, ET, R, the accumulation `A_monthly` is computed from the water balance equation ($P-ET-R$). It is compared with TWSC from GRACE by means of the NSE and cyclostationary metrics.

In [7]:
# Load timeseries of TWS changes
TWSC_monthly=pd.read_csv('../data/hydrology_timeseries/TWS_{}_monthly_filtered.csv'.format(data_TWS),index_col=[0])
TWSC_monthly.columns=TWSC_monthly.columns.astype('datetime64[ns]')
TWSC_monthly=TWSC_monthly[time_idx]

i=0
for data_P in list_data_P:
    P_monthly=pd.read_csv('../data/hydrology_timeseries/P_{}_monthly_filtered.csv'.format(data_P),index_col=[0])
    P_monthly.columns=P_monthly.columns.astype('datetime64[ns]')
    
    for data_ET in list_data_ET:
        ET_monthly=pd.read_csv('../data/hydrology_timeseries/ET_{}_monthly_filtered.csv'.format(data_ET),index_col=[0])
        ET_monthly.columns=ET_monthly.columns.astype('datetime64[ns]')
        
        for data_R in list_data_R:
            R_monthly=pd.read_csv('../data/hydrology_timeseries/R_{}_monthly_filtered.csv'.format(data_R),index_col=[0])
            R_monthly.columns=R_monthly.columns.astype('datetime64[ns]')
            
            i+=1
            if i%10==0:
                print("combination {}/{}".format(i,len(list_combinations)))
            column='P_{}_ET_{}_R_{}_TWS_{}'.format(data_P,data_ET,data_R,data_TWS)
            
            # compute the net accumulation from the water balance equation (A = P - ET - R)
            A_monthly=P_monthly[time_idx]-ET_monthly[time_idx]-R_monthly[time_idx]
            
            # basins where one dataset is missing (for at least one month)
            unknown_basins=basins.index[np.sum(np.isnan(A_monthly),axis=1)!=0]
            
            # RMSD for each basin (without the normalization by the length of the timeseries 
            # because the denominator has the same normalization)
            RMSD=np.sum((A_monthly-TWSC_monthly)**2,axis=1)
            
            # difference between GRACE TWS and the long-term mean (err_cst in the article)
            TWSC_cst=TWSC_monthly.mean(axis=1)
            error_cst_ts=(TWSC_monthly.T-TWSC_cst).T
            error_cst=np.sum(error_cst_ts**2,axis=1)

            # NSE
            NSE=1-RMSD/error_cst
            NSE[unknown_basins]=np.nan
            NSE_tab[column]=NSE.copy()
            
            # to compute the cyclic NSE, we need the mean monthly cycle for each basin
            temp=TWSC_monthly.T
            temp['month']=temp.index.month
            TWSC_monthly_cycle=temp.groupby('month').mean()

            TWSC_cycle=pd.DataFrame(np.zeros((time_idx.shape[0],basins.shape[0]+1)),
                          index=time_idx,
                          columns=list(basins.index)+['month'])
            TWSC_cycle['month']=time_idx.month
            for month in range(1,13):
                TWSC_cycle.loc[TWSC_cycle['month']==month,basins.index]=TWSC_monthly_cycle.loc[month].values
            TWSC_cycle=TWSC_cycle[basins.index].T # transpose to have basins in rows and months in columns

            error_cycle=np.sum((TWSC_monthly-TWSC_cycle)**2,axis=1)
            
            # NSE cyclic
            NSE_cyclic=1-RMSD/error_cycle
            NSE_cyclic[unknown_basins]=np.nan
            NSE_cyclic_tab[column]=NSE_cyclic.copy()
                    

NSE_tab.to_csv("../outputs/NSE_tab_{}basins_{}datasets_{}.csv".format(basins.shape[0],
                                                                                   len(list_combinations),data_TWS))
NSE_cyclic_tab.to_csv("../outputs/NSE_cyclic_tab_{}basins_{}datasets_{}.csv".format(basins.shape[0],
                                                                                   len(list_combinations),data_TWS))

combination 10/1694
combination 20/1694
combination 30/1694
combination 40/1694
combination 50/1694
combination 60/1694
combination 70/1694
combination 80/1694
combination 90/1694
combination 100/1694
combination 110/1694
combination 120/1694
combination 130/1694
combination 140/1694
combination 150/1694
combination 160/1694
combination 170/1694
combination 180/1694
combination 190/1694
combination 200/1694
combination 210/1694
combination 220/1694
combination 230/1694
combination 240/1694
combination 250/1694
combination 260/1694
combination 270/1694
combination 280/1694
combination 290/1694
combination 300/1694
combination 310/1694
combination 320/1694
combination 330/1694
combination 340/1694
combination 350/1694
combination 360/1694
combination 370/1694
combination 380/1694
combination 390/1694
combination 400/1694
combination 410/1694
combination 420/1694
combination 430/1694
combination 440/1694
combination 450/1694
combination 460/1694
combination 470/1694
combination 480/1694
c